# Knowledge Base Completion (KBC) / Link Prediction with RDF2Vec

This notebook allows a fully automated run of RDF2vec link prediction and evaluation.

## Prerequisites (Linux/MacOS)

- install kbc_rdf2vec ([https://github.com/janothan/kbc_rdf2vec](https://github.com/janothan/kbc_rdf2vec))
- install kbc_evaluation ([https://github.com/janothan/kbc_evaluation/](https://github.com/janothan/kbc_evaluation/))
- install jRDF2Vec and add the shell script to your path (see [here](https://github.com/dwslab/jRDF2Vec/blob/master/src/main/bin/jrdf2vec.sh)) - alternatively, you can modify the specified jrdf2vec command in this notebook to fit your path.

The only thing you have to do before running the whole notebook is to set your `work_dir` in the cell below.

In [1]:
# TODO: Now let's decide on your directory where everything shall be written to (requires > 5Gb of disk space)
work_dir = "/work/jportisc/kbc_rdf2vec/strategy_grid_2/evaluation_2"

## Let's Transform WN18 and FB15k Into NT Files

In [2]:
import sys
from kbc_rdf2vec.dataset import DataSet
import os

# create the directory if it does not exist yet
nt_dir = os.path.join(work_dir, "nt_files")
if not os.path.exists(nt_dir):
    os.makedirs(nt_dir)

DataSet.write_training_file_nt(data_set=DataSet.WN18, file_to_write=os.path.join(nt_dir, "WN18.nt"))
DataSet.write_training_file_nt(data_set=DataSet.FB15K, file_to_write=os.path.join(nt_dir, "FB15k.nt"))

## Let's Train Embeddings with jRDF2Vec

Train embeddings for WN18 by running the following line:
```
!jrdf2vec -graph "./WN18.nt" -numberOfWalks 300 -threads 20 -depth 4 -walkDirectory <set manually or use generated statement> -trainingMode sg -dimension 200 -window 2 -epochs 25
```

Train embeddings for FB15k by running the following line:
```
!jrdf2vec -graph "./FB15k.nt" -numberOfWalks 300 -threads 20 -depth 4 -walkDirectory <set manually or use generated statement> -trainingMode sg -dimension 200 -window 2 -epochs 25
```

If you are happy with the default RDF2Vec configuration, you do not have to do anything except for running the cells below.

In [3]:
number_of_walks = 2000
depth = 4
training_mode = "sg"
dimension = 200
window = 5
epochs = 25

configuration_string = f"""
Number of Walks:  {number_of_walks}
Depth:            {depth}
Training Mode:    {training_mode}
Dimension:        {dimension}
Window:           {window}
Epochs:           {epochs}
"""

In [4]:
import os
import io

print(configuration_string)
wn18_walk_path = os.path.join(work_dir, "wn18_walks")
wn_nt_path = os.path.join(nt_dir, "WN18.nt")

!jrdf2vec -graph $wn_nt_path -numberOfWalks $number_of_walks -threads 20 -depth $depth -walkDirectory $wn18_walk_path -trainingMode $training_mode -dimension $dimension -window $window -epochs $epochs

with io.open(os.path.join(wn18_walk_path, "configuration.txt"), 'w+', encoding='utf8') as f:
    f.write(configuration_string)


Number of Walks:  2000
Depth:            4
Training Mode:    sg
Dimension:        200
Window:           5
Epochs:           25

The specified walk directory does not exist. Trying to make the directory.
Using 20 threads for walk generation and training.
Using vector dimension: 200
Using depth 4
Generating 2000 walks per entity.
RDF2Vec Classic
 INFO [main] (ParserManager.java:53) - Using NxParser.
 INFO [main] (ParserManager.java:88) - Model read into memory.
walkGeneration mode is null... Using default: RANDOM_WALKS_DUPLICATE_FREE
 INFO [pool-1-thread-18] (WalkGenerator.java:320) - TOTAL PROCESSED ENTITIES: 1000
 INFO [pool-1-thread-18] (WalkGenerator.java:321) - TOTAL NUMBER OF PATHS : 1303255
 INFO [pool-1-thread-1] (WalkGenerator.java:320) - TOTAL PROCESSED ENTITIES: 2000
 INFO [pool-1-thread-1] (WalkGenerator.java:321) - TOTAL NUMBER OF PATHS : 2616321
 INFO [pool-1-thread-1] (WalkGenerator.java:320) - TOTAL PROCESSED ENTITIES: 3000
 INFO [pool-1-thread-1] (WalkGenerator.java:321

In [ ]:
import os
import io

print(configuration_string)
fb15k_walk_path = os.path.join(work_dir, "fb15k_walks")
fb15k_nt_path = os.path.join(nt_dir, "FB15k.nt")

!jrdf2vec -graph $fb15k_nt_path -numberOfWalks $number_of_walks -threads 20 -depth $depth -walkDirectory $fb15k_walk_path -trainingMode $training_mode -dimension $dimension -window $window -epochs $epochs

with io.open(os.path.join(fb15k_walk_path, "configuration.txt"), 'w+', encoding='utf8') as f:
    f.write(configuration_string)

## Let's Check the Embeddings

```
!jrdf2vec -analyzeVocab ./wn18_walks/model.kv ./WN18.nt &> wn_analysis.txt
!jrdf2vec -analyzeVocab ./fb15k_walks/model.kv ./FB15k.nt &> fb_analysis.txt

```
The reports are written to the specified files (`wn_analysis.txt`/`fb_analysis.txt`) as they can be quite long.
You can find the reports in the walk directories.

In [ ]:
import os

wn18_kv_path = os.path.join(wn18_walk_path, "model.kv")
wn18_analysis_path = os.path.join(wn18_walk_path, "wn_analysis.txt")

!jrdf2vec -analyzeVocab $wn18_kv_path $wn_nt_path &> $wn18_analysis_path

In [ ]:
import os

fb15k_kv_path = os.path.join(fb15k_walk_path, "model.kv")
fb15k_analysis = os.path.join(fb15k_walk_path, "fb_analysis.txt")

!jrdf2vec -analyzeVocab $fb15k_kv_path $fb15k_nt_path &> $fb15k_analysis

## Let's predict!
We start by generating the files containing the predictions.

In [ ]:
from kbc_rdf2vec.dataset import DataSet
from kbc_rdf2vec.prediction import PredictionFunctionEnum, PredictionFunction
from kbc_rdf2vec.rdf2vec_kbc import Rdf2vecKbc

import os


def generate_prediction_files() -> None:
    wn_vector_file = wn18_kv_path
    wn_nt_file = wn_nt_path
    fb15k_vector_file = fb15k_kv_path
    fb15k_nt_file = fb15k_nt_path

    # let's make a directory if it does not exist yet
    prediction_path = os.path.join(work_dir, "predictions")
    if not os.path.exists(prediction_path):
        os.makedirs(prediction_path)
    
    # ANN WN
    kbc = Rdf2vecKbc(
        model_path=wn_vector_file,
        data_set=DataSet.WN18,
        n=None,
        prediction_function=PredictionFunctionEnum.ANN,
        file_for_predicate_exclusion=wn_nt_file,
        is_reflexive_match_allowed=False,
    )
    kbc.predict(os.path.join(prediction_path, "wn_ann.txt"))

    # ANN FB
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        data_set=DataSet.FB15K,
        n=None,
        prediction_function=PredictionFunctionEnum.ANN,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=False,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_ann.txt"))
    

    # most similar WN
    kbc = Rdf2vecKbc(
        model_path=wn_vector_file,
        n=None,
        data_set=DataSet.WN18,
        file_for_predicate_exclusion=wn_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.MOST_SIMILAR,
    )
    kbc.predict(os.path.join(prediction_path, "wn_most_similar.txt"))
    
    # most similar FB
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.MOST_SIMILAR,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_most_similar.txt"))
    
    # avg most similar WN
    kbc = Rdf2vecKbc(
        model_path=wn_vector_file,
        n=None,
        data_set=DataSet.WN18,
        file_for_predicate_exclusion=wn_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.PREDICATE_AVERAGING_MOST_SIMILAR,
    )
    kbc.predict(os.path.join(prediction_path, "wn_averaged_most_similar.txt"))

    # avg most similar FB
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.PREDICATE_AVERAGING_MOST_SIMILAR,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_averaged_most_similar.txt")) 
    
    # addition WN
    kbc = Rdf2vecKbc(
        model_path=wn_vector_file,
        n=None,
        data_set=DataSet.WN18,
        file_for_predicate_exclusion=wn_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "wn_addition.txt"))

    # addition FB
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_addition.txt"))
    
    # addition FB with reflexive matches allowed
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=True,
        prediction_function=PredictionFunctionEnum.ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_reflexive_addition.txt"))
    
    # avg addition WN
    kbc = Rdf2vecKbc(
        model_path=wn_vector_file,
        n=None,
        data_set=DataSet.WN18,
        file_for_predicate_exclusion=wn_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.PREDICATE_AVERAGING_ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "wn_averaged_addition.txt"))

    # avg addition FB
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=False,
        prediction_function=PredictionFunctionEnum.PREDICATE_AVERAGING_ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_averaged_addition.txt"))
    
    # avg addition FB with reflexive matches allowed
    kbc = Rdf2vecKbc(
        model_path=fb15k_vector_file,
        n=None,
        data_set=DataSet.FB15K,
        file_for_predicate_exclusion=fb15k_nt_file,
        is_reflexive_match_allowed=True,
        prediction_function=PredictionFunctionEnum.PREDICATE_AVERAGING_ADDITION,
    )
    kbc.predict(os.path.join(prediction_path, "fb15k_reflexive_averaged_addition.txt"))
    

generate_prediction_files()

## Let's evaluate!
Now we just evaluate the files that we have.

In [ ]:
from kbc_evaluation.evaluator import Evaluator, EvaluatorResult 
from kbc_rdf2vec.dataset import DataSet
from typing import List, Tuple
import os
import pickle


work_dir = "/work/jportisc/kbc_rdf2vec/strategy_grid_2/evaluation_2"
prediction_path = os.path.join(work_dir, "predictions")

def evaluate_files() -> List[Tuple[str, EvaluatorResult]]:
    
    result_map = {}
    
    # Let's make a directory for the evaluation
    evaluation_path = os.path.join(work_dir, "evaluation")
    if not os.path.exists(evaluation_path):
        os.makedirs(evaluation_path)
    
    #--------------------
    # evaluation of WN 18
    #--------------------
    
    file_to_be_written=os.path.join(evaluation_path, "wn_ann_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "wn_ann.txt"),
        data_set=DataSet.WN18,
        n=10,
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["ANN"] = [results]
    
    
    file_to_be_written=os.path.join(evaluation_path, "wn_most_similar_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "wn_most_similar.txt"),
        data_set=DataSet.WN18,
        n=10,
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H, L)"] = [results]

    
    file_to_be_written=os.path.join(evaluation_path, "wn_averaged_most_similar_result.txt")
    Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "wn_averaged_most_similar.txt"),
        data_set=DataSet.WN18,
        n=10,
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H, AVG(T-H))"] = [results]

    
    file_to_be_written=os.path.join(evaluation_path, "wn_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "wn_addition.txt"),
        data_set=DataSet.WN18,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + L)"] = [results]
    
    
    file_to_be_written=os.path.join(evaluation_path, "wn_averaged_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "wn_averaged_addition.txt"),
        data_set=DataSet.WN18,
        n = 10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + AVG(T-H))"] = [results]
    
    
    #--------------------
    # evaluation of fb15k
    #--------------------    

    file_to_be_written=os.path.join(evaluation_path, "fb15k_ann_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_ann.txt"),
        data_set=DataSet.FB15K,
        n=10,
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["ANN"].append(results)

    
    file_to_be_written=os.path.join(evaluation_path, "fb15k_most_similar_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_most_similar.txt"),
        data_set=DataSet.FB15K,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H, L)"].append(results)

    
    file_to_be_written=os.path.join(evaluation_path, "fb15k_averaged_most_similar_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_averaged_most_similar.txt"),
        data_set=DataSet.FB15K,
        n=10,
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H, AVG(T-H))"].append(results)

    
    file_to_be_written=os.path.join(evaluation_path, "fb15k_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_addition.txt"),
        data_set=DataSet.FB15K,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + L)"].append(results)
    

    file_to_be_written=os.path.join(evaluation_path, "fb15k_averaged_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_averaged_addition.txt"),
        data_set=DataSet.FB15K,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + AVG(T-H))"].append(results)
    

    file_to_be_written=os.path.join(evaluation_path, "fb15k_reflexive_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_reflexive_addition.txt"),
        data_set=DataSet.FB15K,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + L) reflexive"] = [None, results]
    

    file_to_be_written=os.path.join(evaluation_path, "fb15k_reflexive_averaged_addition_result.txt")
    results = Evaluator.calculate_results(
        file_to_be_evaluated=os.path.join(prediction_path, "fb15k_reflexive_averaged_addition.txt"),
        data_set=DataSet.FB15K,
        n=10
    )
    Evaluator.write_result_object_to_file(file_to_be_written=file_to_be_written, result_object=results)
    result_map["most_similar(H + AVG(T-H)) reflexive"] = [None, results]
    
    # The result map is expensive to calculate, so let's persist it.
    with open(os.path.join(prediction_path, "result_map.pickle"), 'wb') as f:
        pickle.dump(result_map, f, pickle.HIGHEST_PROTOCOL)
    
    return result_map

    
result_map = evaluate_files()

Reading provided file...
2021-02-01 23:09:47,804 - root - INFO - Hits@10 Heads: 3342
2021-02-01 23:09:47,806 - root - INFO - Hits@10 Tails: 1626
2021-02-01 23:09:47,807 - root - INFO - Hits@10 Total: 4968
2021-02-01 23:09:48,255 - kbc_evaluation.evaluator - INFO - Calculating Mean Rank
2021-02-01 23:09:48,397 - root - INFO - Mean Head Rank: 108.275 (0 ignored lines)
Mean Reciprocal Head Rank: 0.4350056284916208 (0 ignored lines)
2021-02-01 23:09:48,399 - root - INFO - Mean Tail Rank: 598.5542 (0 ignored lines)
Mean Reciprocal Tail Rank: 0.20552996005991991 (0 ignored lines)
2021-02-01 23:09:48,400 - root - INFO - Mean rank: 353.4146; rounded: 353
2021-02-01 23:09:48,401 - root - INFO - Mean reciprocal rank: 0.32026779427577035
Reading provided file...
2021-02-01 23:10:11,344 - kbc_evaluation.dataset - INFO - Apply Filtering
2021-02-01 23:10:11,346 - kbc_evaluation.dataset - INFO - Read Training File
2021-02-01 23:10:34,440 - kbc_evaluation.dataset - INFO - Read Validation File
2021-02-

## Let's Render our Evaluation Results
We have already individual evaluation files written to disk. Now, let's quickly render an HTML table.

In [7]:
from IPython.display import display, HTML
from typing import Dict, List
from kbc_evaluation.evaluator import EvaluatorResult 
import os
import pickle

work_dir = "/work/jportisc/kbc_rdf2vec/strategy_grid_2/evaluation_2"
prediction_path = os.path.join(work_dir, "predictions")

def transform_result_list_to_html(result_map: List[Dict[str, List[EvaluatorResult]]]) -> str:
    if "result_map" not in globals() or result_map is None:
        with open(os.path.join(work_dir, "predictions", "result_map.pickle"), 'rb') as f:
            print("Loading result_map from disk.")
            result_map = pickle.load(f)
    
    first_entry = next(iter(result_map.values()))
    
    if first_entry[0].n is not None:
        n = first_entry[0].n
    elif first_entry[1].n is not None:
        n = first_entry[1].n
    else:
        n = "?"
    
    result = f"""
        <table style="border: 1px solid black;">
            <tr>
                <td>&nbsp;</td>
                <td colspan="6"><center><b>WN18</b></center></td>
                <td colspan="6"><center><b>FB15k</b></center></td>
            </tr>
            <tr>
                <td>Metric</td>
                <td colspan="2"><center>Mean Rank (all)</center></td>
                <td colspan="2"><center>HITS@{n} (all)</center></td>
                <td colspan="2"><center>RelativeHITS@{n} (all)</center></td>
                <td colspan="2"><center>Mean Rank (all)</center></td>
                <td colspan="2"><center>HITS@{n} (all)</center></td>
                <td colspan="2"><center>RelativeHITS@{n} (all)</center></td>
            <tr>
            <tr>
                <td>Evaluation Setting</td>
                <td>Raw</td>
                <td>Filtered</td>
                <td>Raw</td>
                <td>Filtered</td>
                <td>Raw</td>
                <td>Filtered</td>
                <td>Raw</td>
                <td>Filtered</td>
                <td>Raw</td>
                <td>Filtered</td>
                <td>Raw</td>
                <td>Filtered</td>
            </tr>
        """    
    
    for setting, entry in result_map.items():
        result = result + f"""
            <tr>
                <td>{setting}</td>
            """
        if entry[0] is not None:
            result = result + f"""
                <td>{entry[0].non_filtered_mean_rank_all}</td>
                <td>{entry[0].filtered_mean_rank_all}</td>
                <td>{entry[0].non_filtered_hits_at_n_all}</td>
                <td>{entry[0].filtered_hits_at_n_all}</td>
                <td>{round(entry[0].non_filtered_hits_at_n_relative, 4)}</td>
                <td>{round(entry[0].filtered_hits_at_n_relative, 4)}</td>
            """
        else:
            result = result + f"""
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
            """
        if entry [1] is not None:
            result = result + f"""
                <td>{entry[1].non_filtered_mean_rank_all}</td>
                <td>{entry[1].filtered_mean_rank_all}</td>
                <td>{entry[1].non_filtered_hits_at_n_all}</td>
                <td>{entry[1].filtered_hits_at_n_all}</td>
                <td>{round(entry[1].non_filtered_hits_at_n_relative, 4)}</td>
                <td>{round(entry[1].filtered_hits_at_n_relative, 4)}</td>
            """
        else:
             result = result + f"""
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
                <td>&nbsp;</td>
            """
        
        result = result + "</tr>"
    
    result = result + "\n</table>"
    return result


display(HTML(transform_result_list_to_html(None)))


Loading result_map from disk.
